# Evaluate model in paper Machine Learning for Trading
Link [paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3015609)

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_ta as ta
from stable_baselines3.dqn import DQN
import gym
import matplotlib.pyplot as plt
from stable_baselines3.common.evaluation import evaluate_policy

import stock_env
from stock_env.utils import *

%matplotlib inline

In [2]:
from stable_baselines3.common.env_checker import check_env
from stock_env.envs.vn_stock_env import VietnamStockEnv
import mt4_hst

data = mt4_hst.read_hst("stock_env/datasets/FPT1440.hst")
# data.set_index('time', inplace=True)
data.shape

(3888, 6)

In [3]:
data.tail()

,time,open,high,low,close,volume
3883,2022-07-19,82.900002,83.000000,82.199997,83.000000,1185400.0
3884,2022-07-20,85.000000,85.900002,84.000000,84.900002,2848900.0
3885,2022-07-21,85.900002,85.900002,84.500000,85.000000,1804200.0
3886,2022-07-22,86.400002,86.500000,85.099998,85.099998,1307200.0
3887,2022-07-25,87.000000,87.000000,85.099998,85.400002,2245900.0


In [4]:
# help(data.ta)
data.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 